In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.optim as optim
torch.manual_seed(1)

import random
random.seed(1)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w7-p2/sample_submission.csv
/kaggle/input/2021-ai-w7-p2/x_test.csv
/kaggle/input/2021-ai-w7-p2/y_train.csv
/kaggle/input/2021-ai-w7-p2/x_train.csv


In [2]:
X = pd.read_csv('/kaggle/input/2021-ai-w7-p2/x_train.csv', encoding ='CP949')
y = pd.read_csv('/kaggle/input/2021-ai-w7-p2/y_train.csv', encoding ='CP949')
test = pd.read_csv('/kaggle/input/2021-ai-w7-p2/x_test.csv', encoding ='CP949')#.drop(['시도명'], axis = 1)
sample = pd.read_csv('/kaggle/input/2021-ai-w7-p2/sample_submission.csv', encoding ='CP949')
X = X#.drop(['시도명'], axis = 1)

In [3]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sc = StandardScaler()
X_sc = sc.fit_transform(X)
test_sc = sc.transform(test)

#거주인구	근무인구	방문인구	총 유동인구
# X['거주인구'] = sc.fit_transform(X['거주인구'].to_numpy().reshape(-1, 1))
# test['거주인구'] = sc.transform(test['거주인구'].to_numpy().reshape(-1, 1))
# X['근무인구'] = sc.fit_transform(X['근무인구'].to_numpy().reshape(-1, 1))
# test['근무인구'] = sc.transform(test['근무인구'].to_numpy().reshape(-1, 1))
# X['방문인구'] = sc.fit_transform(X['방문인구'].to_numpy().reshape(-1, 1))
# test['방문인구'] = sc.transform(test['방문인구'].to_numpy().reshape(-1, 1))
# X['총 유동인구'] = sc.fit_transform(X['총 유동인구'].to_numpy().reshape(-1, 1))
# test['총 유동인구'] = sc.transform(test['총 유동인구'].to_numpy().reshape(-1, 1))


In [4]:
test.shape

(10583, 14)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device =='cuda':
    torch.cuda.manual_seed_all(1)

In [6]:
X = torch.FloatTensor(X.to_numpy()).to(device)
y = torch.FloatTensor(y.to_numpy()).to(device)
test = torch.FloatTensor(test.to_numpy()).to(device)

X_sc = torch.FloatTensor(X_sc).to(device)
test_sc = torch.FloatTensor(test_sc).to(device)

In [7]:
X.shape

torch.Size([21486, 14])

In [8]:
linear1 = torch.nn.Linear(14, 512)
linear2 = torch.nn.Linear(512, 256)
linear3 = torch.nn.Linear(256, 256)
linear4 = torch.nn.Linear(256, 1)
relu = torch.nn.ReLU()
#dropout = torch.nn.Dropout(0.3)
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)

model = torch.nn.Sequential(linear1, relu, #dropout,
                            linear2, relu, #dropout,
                            linear3, relu, #dropout,
                            linear4).to(device)

In [9]:
model.train()
loss = torch.nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(8001):
    hypothesis = model(X_sc)
    cost = loss(hypothesis, y)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 1000==0:
        print(epoch, cost.item())

0 374.4515380859375
1000 35.58760070800781
2000 31.72509002685547
3000 28.559833526611328
4000 28.15683937072754
5000 25.912555694580078
6000 26.633882522583008
7000 25.66533851623535
8000 23.91303825378418


In [10]:
# model.train()
# loss = torch.nn.L1Loss()
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# for epoch in range(5001):
#     hypothesis = model(X)
#     cost = loss(hypothesis, y)
#     optimizer.zero_grad()
#     cost.backward()
#     optimizer.step()
#     if epoch % 500==0:
#         print(epoch, cost.item())

In [11]:
model.eval()
with torch.no_grad():
    predict = model(test_sc).cpu()
    predict = predict.float().numpy()

In [12]:
sample['predict'] = predict
sample.head(20)

,id,predict
0,0,0.797257
1,1,1.454939
2,2,0.388508
3,3,1116.374146
4,4,1998.790894
5,5,1589.487549
6,6,0.526260
7,7,-0.048144
8,8,1130.505493
9,9,838.224121


In [13]:
sample.to_csv('submit.csv', index=False)